In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath('/home/cneubauer/repos/plio_master/plio'))
sys.path.insert(0, os.path.abspath('/home/cneubauer/repos/autocnet_server'))
sys.path.insert(0, os.path.abspath('/home/cneubauer/repos/autocnet_dev/autocnet'))

os.environ['autocnet_config'] = '/home/cneubauer/autocnet_projects/demo.yml'

%pylab inline
from autocnet_server.graph.graph import NetworkCandidateGraph


Populating the interactive namespace from numpy and matplotlib


/usgs/apps/anaconda/envs/autocnet/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Setup
# Create the candidate graph
ncg = NetworkCandidateGraph.from_database()

ncg.apply('project_keypoints', on='node')

13

In [4]:
# Example function use as baseline

ncg.apply('ring_match', on='edge', ring_radius=30, max_radius=300, target_points=25, tolerance_val=0.1)
# The printed integer is the number of nodes/edges that the algorithm is applied to

baseline = ncg.matches()
del(ncg.matches())


AttributeError: 'NetworkCandidateGraph' object has no attribute 'matches'

In [ ]:
import pandas as pd
import numpy as np

# fuzz the ring_radius decrement first. This can be purely iterative because it is locked in between 30 and 0

# define a thresh hold that for differences to be interesting enough to look at more
# arbitrary pick @ 10%
thresh = .10
interesting_radius = []
data_dir = '/home/cneubauer/data/ring_matcher/'

for radius in range(30, 0, -1):
    ncg.apply('ring_match', on='edge', ring_radius=radius, max_radius=300, target_points=25, tolerance_val=0.1)
    matches = ncg.matches
    # get an intersection to calculate differences
    intersection = pd.merge( baseline, matches, how='inner', on=['source_idx', 'destination_idx'])
    intersection.dropna(inplace=True)
    matches_count = len(matches.index)
    intersection_count = len(intersection.index)
    difference = 1 - ( intersection_count / matches_count )
    
    if ( difference >= thresh ):
        # write out with params so we can get all data with out issue if we fuzz other paramaters as well
        interesting_radius.append( radius )
        matches.to_csv( data_dir + 'interesting_matches/matches_with_r_{}_maxr_{}_tp_{}tv_{}'.format(radius, 300, 25, 0.1) )
        del(ncg.matches)

# output radii that led to interesting outputs        
np.savetxt("radii_list.csv", np.asarray(interesting_radius), delimiter=",")
